# Task 4: RAG检索增强生成管道演示 (无faiss依赖版本)

本演示展示如何使用无faiss依赖的RAG管道进行问答。

## 功能特点
- ✅ 使用numpy实现向量相似度计算
- ✅ 支持文档检索和答案生成
- ✅ 完整的溯源和相似度评分
- ✅ 无需faiss依赖，兼容性更好

In [ ]:
import os
import sys
import json
import time
from typing import Dict, Any, List

# 添加当前目录到Python路径
sys.path.append(os.path.dirname(os.path.abspath('.')))

# 导入自定义模块
from rag_pipeline_no_faiss import RAGPipelineNoFaiss
from llm_api_client import GenerationAPIClient
from embedding_module import EmbeddingAPIClient

print('✓ 依赖导入成功')

## 1. 检查必要文件

确认向量索引和元数据文件是否存在。

In [ ]:
# 定义文件路径
index_file = 'output/document_vectors.npy'
metadata_file = 'output/document_metadata.json'

print('=== 文件检查 ===')

# 检查向量索引文件
if os.path.exists(index_file):
    file_size = os.path.getsize(index_file) / (1024 * 1024)  # MB
    print(f'✅ 向量索引文件存在: {index_file}')
    print(f'📊 文件大小: {file_size:.2f} MB')
else:
    print(f'❌ 向量索引文件不存在: {index_file}')

# 检查元数据文件
if os.path.exists(metadata_file):
    file_size = os.path.getsize(metadata_file) / 1024  # KB
    print(f'✅ 元数据文件存在: {metadata_file}')
    print(f'📊 文件大小: {file_size:.2f} KB')
    
    # 读取元数据统计
    with open(metadata_file, 'r', encoding='utf-8') as f:
        metadata = json.load(f)
    print(f'📋 文档数量: {len(metadata)}')
else:
    print(f'❌ 元数据文件不存在: {metadata_file}')

## 2. 初始化RAG管道

创建RAG管道实例并加载必要的资源。

In [ ]:
print('=== 初始化RAG管道 ===')

try:
    # 初始化RAG管道
    rag_pipeline = RAGPipelineNoFaiss(index_file, metadata_file)
    
    # 获取管道统计信息
    stats = rag_pipeline.get_stats()
    
    print('\n📊 管道统计信息:')
    for key, value in stats.items():
        print(f'  {key}: {value}')
    
    print('\n🎉 RAG管道初始化成功！')
    
except Exception as e:
    print(f'❌ RAG管道初始化失败: {e}')
    raise

## 3. 测试问答功能

使用预定义的测试问题验证RAG管道的功能。

In [ ]:
# 定义测试问题
test_questions = [
    '什么是人工智能？',
    '机器学习的主要方法有哪些？',
    '深度学习和传统机器学习有什么区别？'
]

print('=== 问答测试 ===')

for i, question in enumerate(test_questions, 1):
    print(f'\n--- 测试问题 {i} ---')
    print(f'问题: {question}')
    
    try:
        # 开始计时
        start_time = time.time()
        
        # 调用RAG管道
        result = rag_pipeline.answer_question(
            question=question,
            top_k=3,
            max_context_length=2000
        )
        
        # 计算耗时
        elapsed_time = time.time() - start_time
        
        # 显示结果
        print(f'\n📝 答案:')
        print(result['answer'])
        
        print(f'\n📚 相关来源 ({len(result["sources"])} 个):')
        for j, source in enumerate(result['sources'][:3], 1):
            similarity = source.get('similarity_score', 0)
            source_name = source.get('source', '未知来源')
            print(f'  {j}. {source_name} (相似度: {similarity:.3f})')
        
        print(f'\n⏱️  处理时间: {elapsed_time:.2f}秒')
        
        # 显示检索统计
        if 'retrieval_stats' in result:
            stats = result['retrieval_stats']
            print(f'📊 检索统计:')
            print(f'  - 索引总文档数: {stats.get("total_docs_in_index", 0)}')
            print(f'  - 检索文档数: {stats.get("docs_retrieved", 0)}')
            print(f'  - 最高相似度: {stats.get("top_similarity", 0):.3f}')
        
    except Exception as e:
        print(f'❌ 处理问题时出错: {e}')
        continue

print('\n✅ 问答测试完成！')

## 4. 交互式问答

您可以在下面的代码块中输入自己的问题进行测试。

In [ ]:
# 交互式问答
custom_question = "请在这里输入您的问题"

if custom_question and custom_question != "请在这里输入您的问题":
    print(f'🔍 处理问题: {custom_question}')
    
    try:
        result = rag_pipeline.answer_question(
            question=custom_question,
            top_k=5,
            max_context_length=3000
        )
        
        print(f'\n📝 答案:')
        print(result['answer'])
        
        print(f'\n📚 参考来源:')
        for i, source in enumerate(result['sources'][:5], 1):
            similarity = source.get('similarity_score', 0)
            source_name = source.get('source', '未知来源')
            content_preview = source.get('content', '')[:100] + '...'
            print(f'  {i}. {source_name} (相似度: {similarity:.3f})')
            print(f'     内容预览: {content_preview}')
            print()
        
    except Exception as e:
        print(f'❌ 处理问题时出错: {e}')
else:
    print('💡 请修改上面的 custom_question 变量来测试您自己的问题')

## 5. 性能分析

分析RAG管道的性能表现。

In [ ]:
import numpy as np

print('=== 性能分析 ===')

# 测试多个问题的平均响应时间
performance_questions = [
    '什么是机器学习？',
    '深度学习的应用领域有哪些？',
    '如何评估模型性能？',
    '什么是过拟合？',
    '神经网络的基本结构是什么？'
]

response_times = []
similarity_scores = []

for question in performance_questions:
    start_time = time.time()
    
    try:
        result = rag_pipeline.answer_question(question, top_k=3)
        elapsed_time = time.time() - start_time
        response_times.append(elapsed_time)
        
        # 收集相似度分数
        if result['sources']:
            top_similarity = result['sources'][0].get('similarity_score', 0)
            similarity_scores.append(top_similarity)
        
        print(f'✓ {question[:30]}... - {elapsed_time:.2f}秒')
        
    except Exception as e:
        print(f'✗ {question[:30]}... - 错误: {e}')

# 计算统计信息
if response_times:
    avg_time = np.mean(response_times)
    min_time = np.min(response_times)
    max_time = np.max(response_times)
    
    print(f'\n📊 响应时间统计:')
    print(f'  平均响应时间: {avg_time:.2f}秒')
    print(f'  最快响应时间: {min_time:.2f}秒')
    print(f'  最慢响应时间: {max_time:.2f}秒')

if similarity_scores:
    avg_similarity = np.mean(similarity_scores)
    min_similarity = np.min(similarity_scores)
    max_similarity = np.max(similarity_scores)
    
    print(f'\n🎯 相似度统计:')
    print(f'  平均相似度: {avg_similarity:.3f}')
    print(f'  最低相似度: {min_similarity:.3f}')
    print(f'  最高相似度: {max_similarity:.3f}')

print('\n✅ 性能分析完成！')

## 总结

本演示展示了无faiss依赖的RAG检索增强生成管道的完整功能：

### ✅ 主要功能
- **向量检索**: 使用numpy实现高效的向量相似度计算
- **文档检索**: 基于语义相似度检索相关文档
- **答案生成**: 结合检索到的上下文生成准确答案
- **来源追踪**: 提供详细的来源信息和相似度评分

### 🔧 技术特点
- **兼容性好**: 无需faiss依赖，适用于各种环境
- **性能稳定**: 基于numpy的向量计算，性能可靠
- **易于部署**: 依赖简单，部署方便
- **功能完整**: 支持完整的RAG工作流程

### 📈 应用场景
- 文档问答系统
- 知识库检索
- 智能客服
- 学术研究辅助